In [1]:
#https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/udacity/2_fullyconnected.ipynb

import timeit
script_start_time = timeit.default_timer()


import numpy as np
import os
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

# ********** Logging settings

import logging

logger = logging.getLogger('notMNIST.TF.MLR.training.SDG')

file_log_handler = logging.FileHandler('logfile.log')
logger.addHandler(file_log_handler)

stderr_log_handler = logging.StreamHandler()
logger.addHandler(stderr_log_handler)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_log_handler.setFormatter(formatter)
stderr_log_handler.setFormatter(formatter)

logger.setLevel('DEBUG')

def logInfo(*args):
  logger.info(concatenate(args))

def logDebug(*args):
  logger.debug(concatenate(args))
  
def logError(*args):
  logger.error(concatenate(args))

def concatenate(args):
  return ' '.join(str(v) for v in args)

# ********** End of Logging settings



data_root = '.' # Change me to store data elsewhere


# ******* load  file

logInfo('Loading pickle file...')
start_time = timeit.default_timer()

pickle_file = os.path.join(data_root, 'notMNIST.pickle')

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  logDebug('Training set', train_dataset.shape, train_dataset.dtype, train_labels.shape)
  logDebug('Validation set', valid_dataset.shape, valid_dataset.dtype, valid_labels.shape)
  logDebug('Test set', test_dataset.shape, test_dataset.dtype, test_labels.shape)

logInfo('Pickle file loaded ({:f} sec).'.format(timeit.default_timer() - start_time))

 
# ******* reformat

logInfo('Reformating data...')
start_time = timeit.default_timer()

image_size = 28
num_labels = 10

def reformat(dataset, labels):
  #reshape to (same training examples quantity, 28*28). Type from float64 to float32 (less memory used)
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  # np.arange(num_labels) -> [0,1,2,3,4,5,6,7,8,9]
  # se labels = [1 0 ...] -> labels[:,None] = [[1][0]...]
  # np.arange(num_labels) == labels[:,None]  -> [[False True False ...][True False False ...]...] 
  # .astype(np.float32) -> [[0.0 1.0 0.0 ...][1.0 0.0 0.0 ...]...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
logDebug('Training set', train_dataset.shape, train_labels.shape)
logDebug('Validation set', valid_dataset.shape, valid_labels.shape)
logDebug('Test set', test_dataset.shape, test_labels.shape)

logInfo('data reformatted ({:f} sec).'.format(timeit.default_timer() - start_time))


# ******* build computation graph


logInfo('Building computation graph...')
start_time = timeit.default_timer()

batch_size = 128
logDebug('batch_size', batch_size)

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  learning_rate = 0.05
  logDebug('learning_rate', learning_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


logInfo('Computation graph built ({:f} sec).'.format(timeit.default_timer() - start_time)) 

# ******* training


logInfo('Training...')
start_time = timeit.default_timer()


def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

num_steps = 100001

logDebug('num_steps', num_steps)

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  logInfo("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      logInfo("Minibatch loss at step %d: %f" % (step, l))
      logInfo("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      logInfo("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  logInfo("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


logInfo('Trained ({:f} sec).'.format(timeit.default_timer() - start_time))


# Plot outputs

# non si può disegnare un grafico come quello in questo esempio dove le x corrispondono a uno scalare
# http://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html#sphx-glr-auto-examples-linear-model-plot-ols-py
# qui d2_test_x_dataset ha shape per es. (100, 784) cioè 100 (o quanti impostati) vettori con 784=28x28 valori

# plt.scatter(d2_test_x_dataset, test_y_labels,  color='black')
# plt.plot(d2_test_x_dataset, pred_y_dataset, color='blue', linewidth=3)

# plt.xticks(())
# plt.yticks(())

#si possono disegnare però per ogni label i valori predetti
#quanto più si avvicinano alla linea di inclinazione 1 tanto più sono corretti

# plt.scatter(test_y_labels, pred_y_dataset, color='blue')
# plt.plot([0,10], [0,10], color='black', linewidth=2)

# # plt.grid(True)
# # plt.legend()

# # plt.xticks(())
# # plt.yticks(())

# plt.show()
 

logInfo('Script execution time: {:f} sec.'.format(timeit.default_timer() - script_start_time))

2018-03-04 00:29:05,922 - notMNIST.TF.MLR.training.SDG - INFO - Loading pickle file...
2018-03-04 00:29:07,323 - notMNIST.TF.MLR.training.SDG - DEBUG - Training set (100000, 28, 28) float32 (100000,)
2018-03-04 00:29:07,324 - notMNIST.TF.MLR.training.SDG - DEBUG - Validation set (30000, 28, 28) float32 (30000,)
2018-03-04 00:29:07,325 - notMNIST.TF.MLR.training.SDG - DEBUG - Test set (10000, 28, 28) float32 (10000,)
2018-03-04 00:29:07,327 - notMNIST.TF.MLR.training.SDG - INFO - Pickle file loaded (1.402428 sec).
2018-03-04 00:29:07,329 - notMNIST.TF.MLR.training.SDG - INFO - Reformating data...
2018-03-04 00:29:10,148 - notMNIST.TF.MLR.training.SDG - DEBUG - Training set (100000, 784) (100000, 10)
2018-03-04 00:29:10,149 - notMNIST.TF.MLR.training.SDG - DEBUG - Validation set (30000, 784) (30000, 10)
2018-03-04 00:29:10,151 - notMNIST.TF.MLR.training.SDG - DEBUG - Test set (10000, 784) (10000, 10)
2018-03-04 00:29:10,152 - notMNIST.TF.MLR.training.SDG - INFO - data reformatted (2.8216